In [ ]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

In [3]:
def post_auth_token() -> dict:
    return requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    }).json()

In [17]:
token_credentials = post_auth_token()
access_tkn = token_credentials['access_token']

In [ ]:
headers_saved = get_headers(access_tkn)
headers_top = get_headers(access_tkn)

Run the code from here ⬇️

In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [3]:
import pandas as pd
import os
import numpy as np
import requests
import spotipy
from itertools import chain

In [9]:
USERNAME = os.environ.get('USERNAME')
CLIENT_ID =os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
REDIRECT_URI = os.environ.get('REDIRECT_URI')

AUTH_URL = 'https://accounts.spotify.com/api/token'
SCOPE_READ = 'user-library-read'
SCOPE_TOP = 'user-top-read'
BASE_URL = 'https://api.spotify.com/v1/'

In [5]:
def get_token_scope(scope:str) -> str:
    return spotipy.util.prompt_for_user_token(username=USERNAME,
                                           scope=scope,
                                           client_id=CLIENT_ID,
                                           client_secret=CLIENT_SECRET,
                                           redirect_uri=REDIRECT_URI
                                           )

In [6]:
def get_headers(token) -> dict:
    return {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': 'Bearer {token}'.format(token=token),
        }

In [7]:
def get_request(url:str, headers: dict, params: dict) -> dict:
    try:
        response = requests.get(url, 
                    headers = headers, params = params)
        json = response.json()
        first_result = json['items']
        return first_result
    except:
        return None

In [10]:
token_saved = get_token_scope(SCOPE_READ)
token_top = get_token_scope(SCOPE_TOP)

In [11]:
headers_saved = get_headers(token_saved)
headers_top = get_headers(token_top)

In [12]:
saved_tracks_from_lib = get_request(BASE_URL+f'me/tracks', headers_saved, params = {'market': 'BR'})

In [13]:
top_tracks = get_request(BASE_URL+f'me/top/tracks', headers_top, params = {'time_range': 'medium_term', 'limit': '50'})

In [14]:
top_tracks_long =  get_request(BASE_URL+f'me/top/tracks', headers_top, params = {'time_range': 'long_term', 'limit':'50'})

In [15]:
top_artists = get_request(BASE_URL+f'me/top/artists', headers_top, params = {'time_range': 'medium_term', 'limit': '50'})

In [16]:
top_artists_long = get_request(BASE_URL+f'me/top/artists', headers_top, params = {'time_range': 'long_term', 'limit': '50'})

Every artist is placed as one element in the list returned. This element is a dictionary, and some of the keys are:
- genres
- images
- popularity
- name
- followers
- uri

In [17]:
def create_df_artists(names: list, genres: list, popularities: list, followers: list, columns_artist: list) -> pd.DataFrame:
    
    selected_data = {}

    selected_data[columns_artist[0]] = names
    selected_data[columns_artist[1]] = genres
    selected_data[columns_artist[2]] = popularities
    selected_data[columns_artist[3]] = followers
    
    return pd.DataFrame.from_dict(selected_data)

In [29]:
def get_attributes_from_artists(data: list):
    followers = []
    genres = []
    name = []
    popularity = []
    
    for index in range(len(data)):
        followers.append(data[index]['followers']['total'])
        genres.append(data[index]['genres'])
        name.append(data[index]['name'])
        popularity.append(data[index]['popularity'])
    
    return followers, genres, name, popularity                     

In [36]:
columns_artist = ['name', 'genres', 'popularity', 'followers']

In [13]:
columns_artist = ['name', 'genres', 'popularity', 'followers']

In [37]:
followers_medium, genres_medium, names_medium, popularities_medium = get_attributes_from_artists(top_artists)

In [38]:
top_artists_medium = create_df_artists(names_medium, genres_medium, popularities_medium, followers_medium, columns_artist)
top_artists_medium

,name,genres,popularity,followers
0,RÜFÜS DU SOL,"[australian electropop, indietronica]",73,837002
1,JAY-Z,"[east coast hip hop, hip hop, rap]",86,6614328
2,Paul Kalkbrenner,"[electronica, german techno, leipzig electroni...",66,972682
3,J. Cole,"[conscious hip hop, hip hop, north carolina hi...",89,14664563
4,Klanglos,[dark techno],47,40016
5,Kanye West,"[chicago rap, rap]",96,15322299
6,Jan Blomqvist,"[deep euro house, electronica, minimal techno]",58,159201
7,Jorja Smith,"[pop, r&b, uk contemporary r&b]",76,2361293
8,Drake,"[canadian hip hop, canadian pop, hip hop, rap,...",100,57789722
9,Natti Natasha,"[latin, latin pop, rap latina, reggaeton, trap...",81,6662127


In [39]:
followers_long, genres_long, names_long, popularities_long = get_attributes_from_artist(top_artists_long)

In [40]:
data_top_artists_long = create_df_artists(names_long, genres_long, popularities_long, followers_long, columns_artist)
data_top_artists_long

,name,genres,popularity,followers
0,Lost Frequencies,"[belgian edm, edm, pop, pop dance, tropical ho...",80,2254911
1,Kanye West,"[chicago rap, rap]",96,15322299
2,RÜFÜS DU SOL,"[australian electropop, indietronica]",73,837002
3,Milky Chance,[german pop],75,1543423
4,Nora En Pure,"[deep house, electra, house, progressive house...",66,289354
5,Kygo,"[edm, pop, pop dance, tropical house]",84,7725664
6,Rudimental,"[dance pop, edm, house, pop, pop dance, tropic...",74,1297757
7,Dimitri Vegas & Like Mike,"[belgian dance, belgian edm, big room, dance p...",75,3112469
8,Labrinth,"[indie poptimism, pop]",80,1728362
9,Rihanna,"[barbadian pop, dance pop, pop, pop rap, urban...",90,45231105


Every track is placed as one element in the list returned. This element is a dictionary, and some of the keys are:
- available markets
- duration_ms
- explicit
- album
    - name
    - release date
    - total tracks
    - artists
        - name
- artist
    - name
- name
- is_local
- popularity
- track_number

In [41]:
columns_tracks = ['track_uri', 'track_name', 'duration_ms', 'explicit', 'artist_name', 'popularity', 
                 'album_name', 'album_release_date', 'album_total_tracks', 'album_track_number', 'available_markets']

In [42]:
def get_attributes_from_tracks(data: list):
    track_uri = []
    track_name = []
    duration = []
    explicit = []
    artist_name = []
    popularity = []
    album_name = []
    album_release_date = []
    album_total_tracks = []
    album_track_number = []
    available_markets = []
    
    for index in range(len(data)):
        track_uri.append(data[index]['uri'])
        track_name.append(data[index]['name'])
        duration.append(data[index]['duration_ms'])
        explicit.append(data[index]['explicit'])
        artist_name.append([k['name'] for k in list(chain(data[index]['artists']))])
        popularity.append(data[index]['popularity'])
        album_name.append(data[index]['album']['name'])
        album_release_date.append(data[index]['album']['release_date'])
        album_total_tracks.append(data[index]['album']['total_tracks'])
        album_track_number.append(data[index]['track_number'])
        available_markets.append(data[index]['available_markets'])
    
    return track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets

In [43]:
def create_df_tracks(track_uri: list, track_name: list, duration: list, explicit: list, artist_name: list,
                    popularity: list, album_name: list, album_release_date: list, album_total_tracks:list, 
                    album_track_number: list, available_markets:list) -> pd.DataFrame:
    
    selected_data = {}
    
    selected_data[columns_tracks[0]] = track_uri
    selected_data[columns_tracks[1]] = track_name
    selected_data[columns_tracks[2]] = duration
    selected_data[columns_tracks[3]] = explicit
    selected_data[columns_tracks[4]] = artist_name
    selected_data[columns_tracks[5]] = popularity
    selected_data[columns_tracks[6]] = album_name
    selected_data[columns_tracks[7]] = album_release_date
    selected_data[columns_tracks[8]] = album_total_tracks
    selected_data[columns_tracks[9]] = album_track_number
    selected_data[columns_tracks[10]] = available_markets
    

    return pd.DataFrame.from_dict(selected_data)

In [44]:
track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets = get_attributes_from_tracks(top_tracks_long)

In [45]:
data_top_tracks_long = create_df_tracks(track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets)
data_top_tracks_long

,track_uri,track_name,duration_ms,explicit,artist_name,popularity,album_name,album_release_date,album_total_tracks,album_track_number,available_markets
0,spotify:track:19a3JfW8BQwqHWUMbcqSx8,Famous,196040,True,[Kanye West],73,The Life Of Pablo,2016-06-10,20,4,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
1,spotify:track:7M0fsXo6rpXOKkveYooBob,Goodbye,256786,False,[Apparat],0,The Devil's Walk,2011,10,4,[]
2,spotify:track:4WIpEgkpmeVPTBAO4pm5zy,Clouds,257306,False,[Milky Chance],45,Blossom (Deluxe),2017-03-17,20,5,"[AD, AE, AR, AT, BE, BG, BH, BO, BR, CH, CL, C..."
3,spotify:track:0Hvvs3VDBQzJ6ksSLA7l4G,Demanding Excellence,210837,False,[Labrinth],46,Euphoria (Original Score from the HBO Series),2019-10-04,26,21,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
4,spotify:track:2fZW7ByWe46rGih0FtP2XK,Nothing Left (feat. Will Heard),236682,False,"[Kygo, Will Heard]",51,Cloud Nine,2016-05-13,15,12,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
5,spotify:track:7zcmIz44ipSUMprZKvEeRO,Free My Mind,160645,False,"[Alok, Rooftime, Dubdogz]",63,Free My Mind,2020-02-21,1,1,[BR]
6,spotify:track:695kvY0b7RypIKs9uiRKp5,"Toast to Our Differences (feat. Shungudzo, Pro...",254773,False,"[Rudimental, Shungudzo, Protoje, Hak Baker]",40,Toast to Our Differences (Deluxe Edition),2019-01-25,16,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CH, C..."
7,spotify:track:0elXIl26ZrovBULLTrEGK6,Habits of My Heart - EP Version,210639,False,[Jaymes Young],63,Habits of My Heart,2014-09-28,5,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
8,spotify:track:4f8Mh5wuWHOsfXtzjrJB3t,I THINK,212013,True,"[Tyler, The Creator]",71,IGOR,2019-05-17,12,3,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
9,spotify:track:42KxOIhF6TIiQWb7hbA8I8,Body Talk (Mamooth),207103,False,"[Moguai, Dimitri Vegas & Like Mike, Julian Per...",34,Body Talk (Mammoth),2014-07-17,2,1,[BR]


In [46]:
track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets = get_attributes_from_tracks(top_tracks)

In [47]:
data_top_tracks_medium = create_df_tracks(track_uri, track_name, duration, explicit, artist_name, popularity, album_name, album_release_date, album_total_tracks, album_track_number, available_markets)
data_top_tracks_medium

,track_uri,track_name,duration_ms,explicit,artist_name,popularity,album_name,album_release_date,album_total_tracks,album_track_number,available_markets
0,spotify:track:0wXuerDYiBnERgIpbb3JBR,Redbone,326933,True,[Childish Gambino],82,"""Awaken, My Love!""",2016-12-02,11,6,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
1,spotify:track:24NPziq3dayIMKppeI94Um,Last Night,255706,False,"[Keyshia Cole, Diddy]",62,Just Like You,2007,16,2,"[AD, AR, AT, AU, BE, BG, BO, BR, CH, CL, CO, C..."
2,spotify:track:6t90Z9XkdsHD8xMxro6KRP,Consideration,161066,True,"[Rihanna, SZA]",68,ANTI (Deluxe),2016-01-28,16,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
3,spotify:track:2xUsXYIiK18wSnv2SXFGtj,Invisible - Paul Kalkbrenner Remix,457036,False,"[NTO, Paul Kalkbrenner]",61,Invisible (Paul Kalkbrenner Remix),2021-04-14,1,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
4,spotify:track:54CipIOeFb1lZYFhHi26lo,Claire,287579,False,"[Anyma, Janus Rasmussen, Delhia De France]",50,Claire,2021-07-16,3,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
5,spotify:track:5Pt83OUaK3K9C2if4C5JOk,Gotta Have It,140746,True,"[JAY-Z, Kanye West]",65,Watch The Throne (Deluxe),2011-08-08,16,5,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
6,spotify:track:6rGBks2LevDjAMo5KJq88Y,No Goodbye - Extended Version,390920,False,[Paul Kalkbrenner],50,No Goodbye,2019-07-26,2,2,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
7,spotify:track:4L7jMAP8UcIe309yQmkdcO,Lost,234093,True,[Frank Ocean],72,channel ORANGE (Explicit Version),2012-07-10,17,11,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CH, C..."
8,spotify:track:2wi4d9kls0CWgE4RsghZyR,Rendezvous,260493,False,[RÜFÜS DU SOL],51,Atlas (Light / Dark Deluxe Edition),2013,23,5,"[AE, AR, AT, AU, BH, BO, BR, CA, CH, CL, CO, C..."
9,spotify:track:4slSrbTK1sNK4I1mDYEthf,Rover (feat. DTG),167916,True,"[S1mba, DTG]",72,Rover (feat. DTG),2020-03-04,1,1,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C..."
